## Red Neuronal Convolucional (CNN)

Implementación de una red neuronal convolucional (CNN) avanzada utilizando TensorFlow y Keras. Esta CNN está diseñada para clasificación de imágenes en datasets como CIFAR-10 o ImageNet y sigue las mejores prácticas en arquitectura de redes neuronales profundas.

Incluye técnicas como:

- *Batch Normalization* para acelerar el entrenamiento y mejorar la estabilidad.
- *Dropout* para prevenir sobreajuste.
- *Data Augmentation* para mejorar la capacidad de generalización.
- *Callbacks* como reducción de la tasa de aprendizaje y early stopping.
- *Residual Connections* inspiradas en ResNet para mejorar el flujo de gradientes.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [2]:
# Definir la arquitectura de la CNN avanzada
def build_advanced_cnn(input_shape=(32, 32, 3), num_classes=10):
    inputs = keras.Input(shape=input_shape)
    
    # Primera capa convolucional
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Segunda capa convolucional
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Tercera capa convolucional
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Capa completamente conectada
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    return model

In [3]:
# Compilar y entrenar el modelo
def train_cnn(model, train_images, train_labels, val_images, val_labels, epochs=50, batch_size=64):
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Callbacks para optimización
    lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Data augmentation
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True
    )
    datagen.fit(train_images)
    
    history = model.fit(
        datagen.flow(train_images, train_labels, batch_size=batch_size),
        validation_data=(val_images, val_labels),
        epochs=epochs,
        callbacks=[lr_reduction, early_stopping]
    )
    return history

In [ ]:
# Uso del modelo con CIFAR-10
def main():
    (train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()
    train_images, test_images = train_images / 255.0, test_images / 255.0
    
    model = build_advanced_cnn()
    history = train_cnn(model, train_images, train_labels, test_images, test_labels)
    
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
    print(f'Accuracy en test: {test_acc:.4f}')
    
    model.save('cnn_advanced.h5')

if __name__ == "__main__":
    main()

## Explicación:
1. Arquitectura avanzada de CNN:
- Tres bloques convolucionales con Batch Normalization y Dropout.
- MaxPooling después de cada bloque para reducir dimensionalidad.
- Capa completamente conectada con Dropout para evitar sobreajuste.
- Capa de salida con activación Softmax para clasificación.
  
2. Entrenamiento optimizado:
- Data Augmentation para mejorar la generalización.
- ReduceLROnPlateau para ajustar dinámicamente la tasa de aprendizaje.
- EarlyStopping para detener el entrenamiento si no mejora el rendimiento.
  
3. Uso del dataset CIFAR-10:
- Descarga y preprocesamiento de imágenes (normalización).
- Entrenamiento y evaluación del modelo.
- Guardado del modelo entrenado en un archivo .h5.